# 📊 Executive Dashboard

**Purpose**: High-level KPI summary for Monday stakeholder presentations.

This dashboard consolidates key metrics from all analysis notebooks into a single, presentation-ready view.

---

## 1. Setup & Data Loading

In [3]:
# Install required packages
!pip install pandas openpyxl plotly -q

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option('display.max_columns', None)
print("✅ Libraries loaded!")
print(f"📅 Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

✅ Libraries loaded!
📅 Report Generated: 2025-12-07 21:35


In [4]:
# File path - update this to match your file location
filename = r"C:\Users\bmalaraju\Documents\WP-OP Agent\JIRA-Agent\11.25.WP Orders_25-11-2025_v01.xlsx"
print(f"📁 Using file: {filename}")

📁 Using file: C:\Users\bmalaraju\Documents\WP-OP Agent\JIRA-Agent\11.25.WP Orders_25-11-2025_v01.xlsx


In [5]:
# Load and prepare data
df = pd.read_excel(filename, engine='openpyxl')
print(f"📊 Loaded: {len(df):,} records")

# Column mapping
COL = {
    'status': 'WP Order Status',
    'product': 'Product',
    'customer': 'Customer',
    'quantity': 'WP Quantity',
    'completed': 'WP Completed Qty',
    'requested': 'WP Requested Delivery Date',
    'added': 'Added Date',
    'approved': 'Approved Date',
    'acknowledged': 'Acknowledgement Date',
    'submitted': 'Submitted Date',
    'itd': 'In-Time Delivery',
    'order_id': 'WP Order ID'
}

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\bmalaraju\\Documents\\WP-OP Agent\\JIRA-Agent\\11.25.WP Orders_25-11-2025_v01.xlsx'

In [ ]:
# Calculate all key metrics
TODAY = pd.Timestamp.now().normalize()
TERMINAL = ['Approved', 'Rejected', 'Cancelled']

# Helper function to normalize timezone-aware dates
def parse_date(series):
    """Parse dates and remove timezone info for consistent comparisons."""
    dt = pd.to_datetime(series, errors='coerce')
    if dt.dt.tz is not None:
        dt = dt.dt.tz_localize(None)
    return dt

# Parse data with timezone normalization
df['requested_date'] = parse_date(df.get(COL['requested']))
df['added_date'] = parse_date(df.get(COL['added']))
df['approved_date'] = parse_date(df.get(COL['approved']))
df['acknowledged_date'] = parse_date(df.get(COL['acknowledged']))
df['submitted_date'] = parse_date(df.get(COL['submitted']))

df['quantity'] = pd.to_numeric(df.get(COL['quantity']), errors='coerce').fillna(0)
df['completed'] = pd.to_numeric(df.get(COL['completed']), errors='coerce').fillna(0)

status_col = COL['status']
df['is_terminal'] = df[status_col].isin(TERMINAL) if status_col in df.columns else False
df['is_approved'] = df[status_col] == 'Approved' if status_col in df.columns else False

# SLA metrics
df['has_target'] = df['requested_date'].notna()
df['is_past_due'] = (df['requested_date'] < TODAY) & df['has_target']
df['is_breached'] = df['is_past_due'] & ~df['is_terminal']

# ITD metric
itd_col = COL['itd']
if itd_col in df.columns:
    df['itd_clean'] = df[itd_col].fillna('').astype(str).str.lower().str.strip()
    df['is_on_time'] = df['itd_clean'].isin(['yes', 'y', '1', 'true'])
    df['itd_known'] = df['itd_clean'].isin(['yes', 'y', '1', 'true', 'no', 'n', '0', 'false'])
else:
    df['itd_known'] = False
    df['is_on_time'] = False

# Cycle time - now works because dates are timezone-normalized
df['cycle_time'] = (df['approved_date'] - df['added_date']).dt.days

print("✅ All metrics calculated!")

---

## 2. 📈 Key Performance Indicators

In [ ]:
# Calculate KPIs
total_orders = len(df)

# SLA Compliance
active_with_date = (~df['is_terminal'] & df['has_target']).sum()
breached = int(df['is_breached'].sum())
sla_compliance = 100 - (breached / active_with_date * 100) if active_with_date > 0 else 100

# Completion Rate
total_qty = df['quantity'].sum()
completed_qty = df['completed'].sum()
completion_rate = (completed_qty / total_qty * 100) if total_qty > 0 else 0

# Approval Rate
approved = int(df['is_approved'].sum())
cancelled = int((df[status_col] == 'Cancelled').sum()) if status_col in df.columns else 0
non_cancelled = total_orders - cancelled
approval_rate = (approved / non_cancelled * 100) if non_cancelled > 0 else 0

# On-Time Delivery
itd_total = int(df['itd_known'].sum())
on_time = int(df['is_on_time'].sum()) if 'is_on_time' in df.columns else 0
otd_rate = (on_time / itd_total * 100) if itd_total > 0 else 0

# Avg Cycle Time
avg_cycle = df['cycle_time'].dropna().mean()
if pd.isna(avg_cycle):
    avg_cycle = 0

# Terminal %
terminal_pct = df['is_terminal'].sum() / total_orders * 100

print("\n" + "="*70)
print("📊 EXECUTIVE KPI SUMMARY")
print("="*70)
print(f"\n📋 VOLUME:            {total_orders:,} total orders")
print(f"\n✅ SLA COMPLIANCE:    {sla_compliance:.1f}%  ({breached:,} breached of {active_with_date} active)")
print(f"📦 COMPLETION RATE:   {completion_rate:.1f}%  ({completed_qty:,.0f} of {total_qty:,.0f} qty)")
print(f"👍 APPROVAL RATE:     {approval_rate:.1f}%  ({approved:,} approved)")
print(f"⏱️ ON-TIME DELIVERY:  {otd_rate:.1f}%  ({on_time:,} of {itd_total:,} tracked)")
print(f"🔄 AVG CYCLE TIME:    {avg_cycle:.1f} days")
print(f"🏁 TERMINAL RATE:     {terminal_pct:.1f}%")
print("="*70)

In [ ]:
# KPI Cards
fig = make_subplots(
    rows=2, cols=3,
    specs=[[{'type': 'indicator'}]*3, [{'type': 'indicator'}]*3],
    subplot_titles=['Total Orders', 'SLA Compliance', 'Completion Rate', 
                    'Approval Rate', 'On-Time Delivery', 'Avg Cycle Time']
)

# Row 1
fig.add_trace(go.Indicator(
    mode="number",
    value=total_orders,
    number={'font': {'size': 48, 'color': '#3498DB'}}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=sla_compliance,
    gauge={'axis': {'range': [0, 100]}, 'bar': {'color': '#4ECDC4'},
           'steps': [{'range': [0, 70], 'color': '#FFCDD2'}, {'range': [70, 85], 'color': '#FFF9C4'}, {'range': [85, 100], 'color': '#C8E6C9'}]},
    number={'suffix': '%', 'font': {'size': 32}}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=completion_rate,
    gauge={'axis': {'range': [0, 100]}, 'bar': {'color': '#2ECC71'},
           'steps': [{'range': [0, 50], 'color': '#FFCDD2'}, {'range': [50, 75], 'color': '#FFF9C4'}, {'range': [75, 100], 'color': '#C8E6C9'}]},
    number={'suffix': '%', 'font': {'size': 32}}
), row=1, col=3)

# Row 2
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=approval_rate,
    gauge={'axis': {'range': [0, 100]}, 'bar': {'color': '#9B59B6'},
           'steps': [{'range': [0, 50], 'color': '#FFCDD2'}, {'range': [50, 75], 'color': '#FFF9C4'}, {'range': [75, 100], 'color': '#C8E6C9'}]},
    number={'suffix': '%', 'font': {'size': 32}}
), row=2, col=1)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=otd_rate,
    gauge={'axis': {'range': [0, 100]}, 'bar': {'color': '#E67E22'},
           'steps': [{'range': [0, 70], 'color': '#FFCDD2'}, {'range': [70, 85], 'color': '#FFF9C4'}, {'range': [85, 100], 'color': '#C8E6C9'}]},
    number={'suffix': '%', 'font': {'size': 32}}
), row=2, col=2)

fig.add_trace(go.Indicator(
    mode="number+delta",
    value=avg_cycle,
    delta={'reference': 30, 'decreasing': {'color': 'green'}, 'increasing': {'color': 'red'}},
    number={'suffix': ' days', 'font': {'size': 40}}
), row=2, col=3)

fig.update_layout(height=500, title={'text': '📊 Executive KPI Dashboard', 'x': 0.5, 'font': {'size': 24}})
fig.show()

---

## 3. 📈 Status Overview

In [ ]:
# Status distribution
if status_col in df.columns:
    status_counts = df[status_col].value_counts()
    
    colors = {
        'Approved': '#4ECDC4', 'Rejected': '#FF6B6B', 'Cancelled': '#95A5A6',
        'Objected': '#FFE66D', 'Pending Acknowledgement': '#3498DB',
        'Acknowledge': '#2ECC71', 'Acknowledged': '#2ECC71',
        'Waiting for order submission': '#9B59B6', 'Waiting for Order Submission': '#9B59B6',
        'Pending Approval': '#E67E22'
    }
    
    fig = go.Figure(data=[go.Pie(
        labels=status_counts.index,
        values=status_counts.values,
        hole=0.5,
        marker_colors=[colors.get(s, '#BDC3C7') for s in status_counts.index],
        textinfo='percent+label',
        textposition='outside'
    )])
    
    fig.update_layout(
        title={'text': 'Order Status Distribution', 'x': 0.5, 'font': {'size': 20}},
        annotations=[{'text': f'{total_orders:,}<br>Orders', 'x': 0.5, 'y': 0.5, 'font_size': 18, 'showarrow': False}],
        height=450,
        showlegend=False
    )
    
    fig.show()

---

## 4. 📊 Top Products & Customers

In [ ]:
# Top 5 Products and Customers side by side
product_col = COL['product']
customer_col = COL['customer']

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Top 5 Products by Volume', 'Top 5 Customers by Volume']
)

# Products
if product_col in df.columns:
    top_products = df[product_col].value_counts().head(5)
    fig.add_trace(go.Bar(
        y=top_products.index,
        x=top_products.values,
        orientation='h',
        marker_color='#3498DB',
        text=top_products.values,
        textposition='outside',
        name='Products'
    ), row=1, col=1)

# Customers
if customer_col in df.columns:
    top_customers = df[customer_col].value_counts().head(5)
    fig.add_trace(go.Bar(
        y=top_customers.index,
        x=top_customers.values,
        orientation='h',
        marker_color='#9B59B6',
        text=top_customers.values,
        textposition='outside',
        name='Customers'
    ), row=1, col=2)

fig.update_layout(
    height=350,
    showlegend=False,
    margin={'l': 150}
)
fig.update_yaxes(categoryorder='total ascending')

fig.show()

---

## 5. 📈 Weekly Trends

In [ ]:
# 8-week volume trend
trend_df = df[df['added_date'].notna()].copy()
trend_df['week'] = trend_df['added_date'].dt.to_period('W').dt.start_time

weekly = trend_df.groupby('week').agg(
    orders=('quantity', 'count'),
    breached=('is_breached', 'sum')
).reset_index().tail(8)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=weekly['week'], y=weekly['orders'], name='Orders', marker_color='#3498DB', opacity=0.7),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=weekly['week'], y=weekly['breached'], name='SLA Breached', 
               mode='lines+markers', line=dict(color='#FF6B6B', width=3)),
    secondary_y=True
)

fig.update_layout(
    title={'text': 'Weekly Order Volume & SLA Breaches', 'x': 0.5, 'font': {'size': 20}},
    height=350,
    legend={'orientation': 'h', 'y': 1.1}
)
fig.update_yaxes(title_text="Orders", secondary_y=False)
fig.update_yaxes(title_text="SLA Breached", secondary_y=True)

fig.show()

---

## 6. 🚨 Attention Required

In [ ]:
# Critical items
at_risk = int((
    (df['requested_date'] >= TODAY) & 
    (df['requested_date'] <= TODAY + timedelta(days=7)) &
    ~df['is_terminal'] & 
    df['has_target']
).sum())

rejected = int((df[status_col] == 'Rejected').sum()) if status_col in df.columns else 0
objected = int((df[status_col] == 'Objected').sum()) if status_col in df.columns else 0

print("\n🚨 ATTENTION REQUIRED")
print("="*50)
print(f"   🔴 SLA Breached (Past Due):     {breached:,}")
print(f"   🟡 At Risk (Due in 7 days):     {at_risk:,}")
print(f"   ❌ Rejected Orders:             {rejected:,}")
print(f"   ⚠️ Objected Orders:             {objected:,}")
print("="*50)

In [ ]:
# Attention indicators
fig = go.Figure()

categories = ['SLA Breached', 'At Risk', 'Rejected', 'Objected']
values = [breached, at_risk, rejected, objected]
colors = ['#FF6B6B', '#FFE66D', '#E74C3C', '#F39C12']

fig.add_trace(go.Bar(
    x=categories,
    y=values,
    marker_color=colors,
    text=values,
    textposition='outside',
    textfont={'size': 18}
))

fig.update_layout(
    title={'text': '🚨 Items Requiring Attention', 'x': 0.5, 'font': {'size': 20}},
    yaxis_title='Count',
    height=350
)

fig.show()

---

## 7. 📄 Export Executive Summary

In [ ]:
# Export summary
export_filename = f"executive_summary_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"

# Create summary dataframe
kpi_df = pd.DataFrame({
    'KPI': ['Total Orders', 'SLA Compliance', 'Completion Rate', 'Approval Rate', 
            'On-Time Delivery', 'Avg Cycle Time', 'SLA Breached', 'At Risk', 'Rejected', 'Objected'],
    'Value': [f"{total_orders:,}", f"{sla_compliance:.1f}%", f"{completion_rate:.1f}%", 
              f"{approval_rate:.1f}%", f"{otd_rate:.1f}%", f"{avg_cycle:.1f} days",
              f"{breached:,}", f"{at_risk:,}", f"{rejected:,}", f"{objected:,}"],
    'Category': ['Volume', 'Quality', 'Progress', 'Progress', 'Quality', 'Efficiency',
                 'Risk', 'Risk', 'Risk', 'Risk']
})

with pd.ExcelWriter(export_filename, engine='openpyxl') as writer:
    kpi_df.to_excel(writer, sheet_name='Executive Summary', index=False)
    
    # Status breakdown
    if status_col in df.columns:
        status_df = df[status_col].value_counts().reset_index()
        status_df.columns = ['Status', 'Count']
        status_df['Percentage'] = (status_df['Count'] / total_orders * 100).round(1)
        status_df.to_excel(writer, sheet_name='Status Breakdown', index=False)

print(f"\n✅ Executive summary exported to: {export_filename}")
# files.download() - uncomment if using Colab
# files.download(export_filename)

---

## 📋 Dashboard Summary

| Category | KPI | Status |
|----------|-----|--------|
| **Volume** | Total Orders | Key volume indicator |
| **Quality** | SLA Compliance | Target: >85% |
| **Quality** | On-Time Delivery | Target: >90% |
| **Progress** | Completion Rate | Qty completed vs ordered |
| **Progress** | Approval Rate | Non-cancelled approved % |
| **Efficiency** | Avg Cycle Time | Days from added to terminal |
| **Risk** | SLA Breached | Immediate attention needed |
| **Risk** | At Risk | Due within 7 days |

### 📌 For Monday's Call
1. **Lead with KPIs**: Start with the 6-gauge dashboard
2. **Highlight Trends**: Show weekly volume chart
3. **Address Risks**: Review attention-required items
4. **Deep Dive**: Use individual notebooks for detailed analysis